In [ ]:
conda install -c conda-forge/label/cf202003 scikit-surprise

In [ ]:
import pandas as pd
from surprise import Dataset
from surprise import Reader

In [ ]:
ratings_dict = {
    "item": [1, 2, 1, 2, 1, 2, 1, 2, 1],
    "user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3],
}

df = pd.DataFrame(ratings_dict)

In [ ]:
df

In [ ]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)

In [ ]:
from surprise import KNNWithMeans

In [ ]:
sim_options = {
    "name": "cosine",
}
algo = KNNWithMeans(k = 3, sim_options=sim_options)

In [ ]:
trainingSet = data.build_full_trainset()

algo.fit(trainingSet)

In [ ]:
prediction = algo.predict('E', 2)
prediction.est

# **Movie Lens**

In [ ]:
import numpy as np

import sklearn


In [ ]:
ratings = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
ratings.head()

In [ ]:
movies = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")
movies.head()

In [ ]:
movies.tail()

In [ ]:
n_ratings = len(ratings)
n_ratings

In [ ]:
n_movies = len(ratings['movieId'].unique())
n_movies

In [ ]:
n_users = len(ratings['userId'].unique())
n_users

In [ ]:

mean_rating = ratings.groupby('movieId')[['rating']].mean()

In [ ]:
mean_rating.head()

In [ ]:
highest_rated = mean_rating['rating'].idxmax()

In [ ]:
movies.loc[movies['movieId'] == highest_rated]

In [ ]:
# Lowest rated movies
lowest_rated = mean_rating['rating'].idxmin()
movies.loc[movies['movieId'] == lowest_rated]

In [ ]:
ratings[ratings['movieId']==highest_rated]

In [ ]:
ratings[ratings['movieId']==lowest_rated]

In [ ]:
N = len(ratings['userId'].unique())
M = len(ratings['movieId'].unique())

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
# Map Ids to indices
user_mapper = dict(zip(np.unique(ratings["userId"]), list(range(N))))
movie_mapper = dict(zip(np.unique(ratings["movieId"]), list(range(M))))

# Map indices to IDs
user_inv_mapper = dict(zip(list(range(N)), np.unique(ratings["userId"])))
movie_inv_mapper = dict(zip(list(range(M)), np.unique(ratings["movieId"])))

user_index = [user_mapper[i] for i in ratings['userId']]
movie_index = [movie_mapper[i] for i in ratings['movieId']]
X = csr_matrix((ratings["rating"], (movie_index, user_index)), shape=(M, N))

In [ ]:
X.shape

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
def find_similar_movies(movie_id, X, k):
      
    neighbour_ids = []
      
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric='cosine')
    kNN.fit(X)

    movie_vec = movie_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [ ]:
movie_titles = dict(zip(movies['movieId'], movies['title']))
  
movie_id = 3
  
similar_ids = find_similar_movies(movie_id, X, k=10)
movie_title = movie_titles[movie_id]
  
print(f"Since you watched {movie_title}")
for i in similar_ids:
    print(movie_titles[i])